<a href="https://colab.research.google.com/github/rogeliorv/encode-ai-bootcamp-g11/blob/main/Gethsun_WeekendI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Interactive AI Chef: Recipe Suggestions, Critiques, and Ingredient-Based Dish Ideas

*This Google Colab notebook provides an interactive AI chef experience. Users can input ingredients to get dish suggestions, request detailed recipes for specific dishes, or submit their own recipes for critique and improvement. The AI, modeled as a young and spirited Kenyan cook, offers clear, detailed, and helpful responses to enhance your cooking experience.*

In [ ]:
!pip install openai

In [22]:
from openai import OpenAI
from google.colab import userdata # Import the userdata module from Google Colab


# Retrieve your API key (replace 'OPEN_API_KEY' with the name you used to store it)
api_key = userdata.get('OPEN_API_KEY')

# Initialize the OpenAI client with the API key
client = OpenAI(api_key=api_key)

In [31]:
# Initial system messages to set the context and behavior of the AI
system_messages = [
    {
        "role": "system",
        "content": (
            "You are a young and spirited Kenyan cook that loves to make Githeri. "
            "You help people by suggesting detailed recipes for dishes they want to cook, "
            "suggesting dishes based on ingredients provided, and criticizing recipes given by the user. "
            "You always try to be as clear as possible and provide the best possible suggestions for the user's needs. "
            "You know a lot about different cuisines and cooking techniques. You are also very patient and understanding with the user's needs and questions."
        ),
    },
    {
        "role": "system",
        "content": (
            "Your client will provide either ingredients, a dish name, or a recipe. "
            "If the input is a list of ingredients, suggest a dish name that can be made with those ingredients, but do not provide the recipe at this stage. "
            "If the input is a dish name, provide a detailed recipe for that dish. "
            "If the input is a recipe, critique the recipe and suggest improvements. "
            "If the input does not match these scenarios, deny the request and ask the user to try again."
        ),
    }
]

# Function to handle ingredients input
def handle_ingredients(input_str):
    messages = system_messages + [
        {
            "role": "user",
            "content": f"Suggest a dish name that can be made with these ingredients: {input_str}"
        }
    ]
    return get_response(messages)

# Function to handle dish name input
def handle_dish(input_str):
    messages = system_messages + [
        {
            "role": "user",
            "content": f"Suggest me a detailed recipe and the preparation steps for making {input_str}"
        }
    ]
    return get_response(messages)

# Function to handle recipe input
def handle_recipe(input_str):
    messages = system_messages + [
        {
            "role": "user",
            "content": f"Criticize this recipe and suggest changes: {input_str}"
        }
    ]
    return get_response(messages)

# Function to get response from OpenAI
def get_response(messages):
    model = "gpt-3.5-turbo"
    stream = client.chat.completions.create(
        model=model,
        messages=messages,
        stream=True,
    )
    collected_messages = []
    for chunk in stream:
        chunk_message = chunk.choices[0].delta.content or ""
        print(chunk_message, end="")
        collected_messages.append(chunk_message)
    return "".join(collected_messages)

# Main function to process user input and call appropriate function
def main():
    user_input = input("Enter your request (ingredients, dish name, or recipe):\n")
    if ',' in user_input:
        print(handle_ingredients(user_input))
    elif any(char.isdigit() for char in user_input):
        print(handle_recipe(user_input))
    else:
        print(handle_dish(user_input))

# Call the main function directly to execute the script
main()


Enter your request (ingredients, dish name, or recipe):
Githeri
Githeri is a popular Kenyan dish made with a mix of maize and beans cooked together. Here's a detailed recipe and preparation steps for making Githeri:

Ingredients:
- 2 cups of maize, soaked overnight
- 1 cup of beans (you can use any beans of your choice), soaked overnight
- 1 onion, chopped
- 2 tomatoes, chopped
- 2 cloves of garlic, minced
- 1 teaspoon of turmeric powder
- 1 teaspoon of cumin powder
- Salt to taste
- Water
- Optional: chopped fresh coriander for garnish

Preparation steps:
1. In a large pot, combine the soaked maize and beans. Add enough water to cover the maize and beans, then bring to a boil.
2. Reduce the heat to low and let it simmer for about 30 minutes or until the maize and beans are partially cooked.
3. In a separate pan, heat some oil over medium heat. Add the chopped onions and sauté until they are translucent.
4. Add the minced garlic and cook for another minute until fragrant.
5. Add the ch

In [32]:
main()

Enter your request (ingredients, dish name, or recipe):
Lamb
I'd be happy to help you with a detailed recipe for preparing lamb. Here's a recipe for a classic Roast Leg of Lamb:

Ingredients:
- 1 leg of lamb (about 5-7 lbs)
- 4-5 cloves of garlic, sliced
- 2-3 sprigs of fresh rosemary
- 2-3 sprigs of fresh thyme
- Salt and pepper to taste
- Olive oil
- 1 cup of lamb or beef broth

Instructions:
1. Preheat your oven to 350°F (175°C).
2. Using a sharp knife, make small incisions all over the lamb.
3. Insert the garlic slices and small sprigs of rosemary and thyme into the incisions.
4. Rub the lamb with olive oil, ensuring it's coated on all sides.
5. Season the lamb generously with salt and pepper.
6. Place the lamb in a roasting pan and add the broth to the pan.
7. Roast the lamb in the preheated oven for about 20 minutes per pound for medium-rare doneness. Use a meat thermometer to check for an internal temperature of 145°F (63°C) for medium-rare, 160°F (71°C) for medium, or 170°F (77

In [33]:
main()

Enter your request (ingredients, dish name, or recipe):
Ingredients: 2 cups maize flour (cornmeal) 4 cups water Salt to taste 1 bunch sukuma wiki (collard greens) 2 tomatoes, chopped 1 onion, chopped Cooking oil
With the ingredients you've provided, you can make a classic Kenyan dish called "Ugali with Sukuma Wiki and Tomato Salsa." It's a delicious and nutritious meal that is popular in Kenya. Let me know if you would like the recipe for this dish!With the ingredients you've provided, you can make a classic Kenyan dish called "Ugali with Sukuma Wiki and Tomato Salsa." It's a delicious and nutritious meal that is popular in Kenya. Let me know if you would like the recipe for this dish!


In [34]:
main()

Enter your request (ingredients, dish name, or recipe):
Milk
I can definitely provide you with a recipe for making milk at home. Here is the recipe and detailed steps:

**Homemade Milk Recipe:**

**Ingredients:**
- 1 cup of whole milk
- 1 tbsp of sugar (optional)
- 1/2 tsp of vanilla extract (optional)

**Instructions:**
1. Pour the whole milk into a small saucepan and place it on the stove over medium heat.
2. Heat the milk slowly, stirring occasionally to prevent it from scorching at the bottom of the pan.
3. If you want sweetened milk, add sugar to the milk and stir until it dissolves.
4. For flavored milk, add vanilla extract to the milk and stir to mix it in.
5. Heat the milk until it reaches your desired temperature. Be careful not to let it boil.
6. Once the milk is heated, remove it from the stove.
7. Let the milk cool slightly before serving.

Enjoy your homemade milk! Let me know if you need any more assistance.I can definitely provide you with a recipe for making milk at hom

In [35]:
main()

Enter your request (ingredients, dish name, or recipe):
Meat
To prepare a simple and delicious meat dish, here's a recipe for Kenyan Nyama Choma (Grilled Meat):

Ingredients:
- 2 pounds of beef or goat meat, cut into large chunks
- 4 cloves of garlic, minced
- 1 tablespoon of ginger, grated
- 1 tablespoon of vegetable oil
- 1 teaspoon of cayenne pepper
- 1 teaspoon of paprika
- Salt to taste
- Freshly ground black pepper
- Lemon wedges for serving

Instructions:
1. In a bowl, mix the minced garlic, grated ginger, vegetable oil, cayenne pepper, paprika, salt, and black pepper to make a marinade.
2. Rub the marinade all over the meat chunks, ensuring they are well coated. Cover the bowl and let it marinate in the refrigerator for at least 1-2 hours, or preferably overnight for the flavors to infuse.
3. Preheat your grill to medium-high heat. If using a charcoal grill, make sure the coals are hot and have turned gray.
4. Remove the marinated meat from the refrigerator and bring it to room

In [36]:
main()

Enter your request (ingredients, dish name, or recipe):
flour
I'm happy to help you with that! Here's a detailed recipe to make flour at home using whole wheat grains:

**Homemade Whole Wheat Flour**

**Ingredients:**
- Whole wheat grains

**Equipment:**
- Grain mill or high-speed blender (such as Vitamix) with a dry container

**Instructions:**
1. **Prepare the Grains:** Before grinding the grains, it's essential to clean them, removing any debris or stones.
   
2. **Grind the Grains:** If you're using a grain mill, follow the manufacturer's instructions for grinding whole grains to make flour. If you're using a high-speed blender, add the grains to the dry container and blend on high until you achieve a fine powder-like consistency.

3. **Sift (optional):** If you want a finer flour, you can sift the ground flour through a fine-mesh sieve to remove any larger particles. This step is optional but can result in a smoother texture.

4. **Store the Flour:** Once you've ground the flour, 